# Instalação e Configuração

> Importante: Caso dê algum erro no processo de instalação e que impeça de prosseguir com a execução do código, confira o Colab da aula e verifique se fez uma cópia do mais atualizado, pois atualizaremos essas etapas de instalação com os comandos atualizados (caso seja necessária alguma mudança no comando de instalação).

As bibliotecas abaixo são essenciais para o desenvolvimento de modelos de aprendizado de máquina e processamento de linguagem natural (NLP). *Transformers*, da HuggingFace, oferece uma vasta gama de modelos pré-treinados como BERT, GPT e T5 para tarefas de NLP. *Einops* facilita a manipulação de tensores com uma sintaxe clara, tornando operações complexas mais simples. *Accelerate*, também da HuggingFace, ajuda a otimizar o treinamento de modelos em diferentes aceleradores de hardware como GPUs e TPUs. Por fim, *BitsAndBytes* possibilita a quantização eficiente de modelos grandes, reduzindo o consumo de memória em PyTorch.

In [ ]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.5 MB/s eta 0:00:00


Em nossa próxima célula, configuraremos o ambiente importando as bibliotecas necessárias e configurando nosso dispositivo.

Vamos importar alguns componentes da biblioteca transformers

* AutoModelForCausalLM: Uma classe que fornece um modelo de linguagem causal (ou autoregressivo) pré-treinado (por exemplo, GPT-2, GPT-3) que são adequados para tarefas de geração de texto.
* AutoTokenizer: Uma classe que fornece um tokenizador que corresponde ao modelo. O tokenizador é responsável por converter texto em tokens (numéricos) que o modelo pode entender.
* pipeline: fornece uma interface simples e unificada para várias tarefas de PNL, facilitando a execução de tarefas como geração de texto, classificação e tradução.
* BitsAndBytesConfig: Uma classe para configuração de quantização e outras otimizações de baixo nível para melhorar a eficiência computacional.



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

Também estamos definindo a variável device, que especifica o dispositivo de computação a ser usado:

* Esta linha verifica se uma GPU habilitada para CUDA está disponível. Se estiver, o código define o dispositivo para cuda:0 (a primeira GPU). Se não estiver, ele volta a usar a CPU.

Lembrando que o uso de GPU pode acelerar significativamente o treinamento e a inferência de modelos de aprendizado profundo. Vamos aproveitar da GPU gratuita do Colab (T4).

In [ ]:
import torch
import getpass
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda:0'

Embora não seja necessário, você também pode definir uma seed / semente, para garantir reprodutibilidade entre diferentes experimentos e execuções.

Assim, podemos garantir que os mesmos números aleatórios sejam gerados sempre que o código for executado, levando a resultados consistentes.

In [ ]:
torch.random.manual_seed(42)

## Definição do token

Abaixo você deve colar o seu token gerado dentro do painel da Hugging Face

In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

# você também pode deixar escrito direto no código para facilitar reexecuções futuras
# só tome cuidado se for compartilhar seu código em algum local, pois você nunca deve deixar suas chaves expostas (principalmente se for de uma API paga)
#os.environ["HF_TOKEN"] = "hf_..."

··········


## Carregando o Modelo

Nesta etapa, faremos o download e a configuração de um modelo do HuggingFace. Este processo pode levar alguns minutos, pois o modelo tem alguns GB - mas no geral o download no Colab deve ser relativamente rápido.


Primeiro vamos começar mostrando o Phi 3 (microsoft/Phi-3-mini-4k-instruct), um modelo menor mas que demonstrou ser muito interessante e comparável a outros muito maiores.

https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

Foi escolhido porque é open source, acessível e consegue responder bem em português (embora ainda seja melhor em inglês). Verá que muitos modelos não entendem esse idioma, e os que entendem são muito pesados para executarmos em nosso ambiente, ou seja, precisamos acessar via alguma API ou interface web, como o ChatGPT. Porém nesse momento queremos explorar soluções open source, para obtermos maior liberdade.


> [checar slides para conferir descrição]

Como não sabemos quando estará vendo essa aula, sugiro conferir leaderboard para obter modelos mais atualizados, já que a competição está bem alta e novos são lançados a todo instante


In [ ]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

* `device_map="cuda"`: Especifica que o modelo deve ser carregado em uma GPU habilitada para CUDA. Lembrando que essa é uma das principais vantagens de estarmos usando agora o Colab pois GPU melhora significativamente o desempenho da inferência e do treinamento do modelo ao alavancar o processamento paralelo.

* `torch_dtype="auto"`: Define automaticamente o tipo de dados apropriado para os tensores do modelo. Isso garante que o modelo use o melhor tipo de dados para desempenho e eficiência de memória, geralmente float32 ou float16.

* `trust_remote_code=True`: Permite o carregamento de código personalizado do repositório de modelos no HuggingFace. Isso é necessário para certos modelos que exigem configurações ou implementações específicas não incluídas na biblioteca padrão.

* `attn_implementation="eager"`: Especifica o método de implementação para o mecanismo de atenção. A configuração "eager" é uma implementação particular que pode fornecer melhor desempenho para alguns modelos ao processar o mecanismo de atenção de uma maneira específica.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
    attn_implementation="eager"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

## Tokenizer

Em nossa configuração, também precisamos carregar o tokenizer associado ao modelo. O tokenizer é crucial para preparar dados de texto em um formato que o modelo possa entender.

* Um tokenizador converte texto bruto em tokens, que são representações numéricas que o modelo pode processar. Ele também converte os tokens de saída do modelo de volta em texto legível por humanos.
* Os tokenizadores lidam com tarefas como dividir texto em palavras ou subpalavras, adicionar tokens especiais e gerenciar mapeamento de vocabulário.

> [mais descrição nos slides]


O tokenizador é um componente crucial no pipeline de PNL, preenchendo a lacuna entre o texto bruto e os tokens prontos para o modelo.

Para implementar, usaremos a função `AutoTokenizer.from_pretrained()`, especificando o mesmo tokenizer do modelo, garantindo assim a consistência entre o processamento de texto durante o treinamento e a inferência.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Criação do Pipeline

Agora criaremos um pipeline para geração de texto usando nosso modelo e tokenizer carregados anteriormente. A função de pipeline HuggingFace simplifica o processo de execução de várias tarefas de processamento de linguagem natural ao fornecer uma interface de alto nível.

Um pipeline é uma abstração que simplifica o uso de modelos pré-treinados para uma variedade de tarefas de PNL. Ele fornece uma API unificada para diferentes tarefas, como geração de texto, classificação de texto, tradução e muito mais.


> [mais descrição nos slides]

Parâmetros:

* `"text-generation"`: especifica a tarefa que o pipeline está configurado para executar. Neste caso, estamos configurando um pipeline para geração de texto. O pipeline usará o modelo para gerar texto com base em um prompt fornecido.
* `model=model`: especifica o modelo pré-treinado que o pipeline usará. Aqui, estamos passando o model que carregamos anteriormente. Este modelo é responsável por gerar texto com base nos tokens de entrada.
* `tokenizer=tokenizer`: especifica o tokenizador que o pipeline usará. Passamos o tokenizer que carregamos anteriormente para garantir que o texto de entrada seja tokenizado corretamente e os tokens de saída sejam decodificados com precisão.




In [ ]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

## Parâmetros para geração de texto

Para personalizar o comportamento do nosso pipeline de geração de texto, podemos passar um dicionário de argumentos para controlar vários aspectos do processo de geração.

`max_new_tokens` - Este parâmetro especifica o número máximo de novos tokens a serem gerados em resposta ao prompt de entrada. Ele controla o comprimento do texto gerado.
* Exemplo: Definir max_new_tokens como 500 significa que o modelo gerará até 500 tokens além do prompt de entrada.

`return_full_text` - Determina se deve retornar o texto completo, incluindo o prompt de entrada, ou apenas os tokens recém-gerados.
* Exemplo: Definir return_full_text como False significa que apenas os tokens recém-gerados serão retornados, excluindo o prompt de entrada original. Se definido como True, o texto retornado incluirá o prompt de entrada e a continuação gerada.

`temperature` - Controla a aleatoriedade do processo de geração de texto. Valores mais baixos tornam a saída do modelo mais determinística e focada, enquanto valores mais altos aumentam a aleatoriedade e a criatividade.
* Exemplo: Uma temperatura de 0,1 torna as previsões do modelo mais confiáveis e menos variadas, levando a saídas mais previsíveis. Uma temperatura mais alta resultaria em um texto mais diverso e variado.

`do_sample` - Este parâmetro habilita ou desabilita a amostragem (sampling) durante a geração de texto. Quando definido como True, o modelo faz a amostragem de tokens com base em suas probabilidades, adicionando um elemento de aleatoriedade à saída. Quando definido como False, o modelo sempre escolhe o token de maior probabilidade (decodificação gananciosa / greedy decoding).
* Exemplo: Definir do_sample como True permite uma geração de texto mais diversa e criativa. Se definido como False, a saída será mais determinística, mas potencialmente menos interessante.


In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True,
}

Gerando a saída: output = pipe(messages, **generation_args): Esta linha passa a mensagem de entrada e os argumentos de geração para o pipeline de geração de texto. O pipeline gera uma resposta com base na mensagem de entrada e nos parâmetros especificados.

* `**generation_args`: Isso descompacta o dicionário generation_args e passa seu conteúdo como argumentos de palavra-chave para o pipeline, personalizando o processo de geração de texto.

In [ ]:
prompt = "Explique o que é computação quântica"
#prompt = "Quanto é 7 x 6 - 42?"

output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


In [ ]:
output

[{'generated_text': ' e como ela difere da computação clássica.\n\n\n### Solution:\n\nA computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os qubits realizem múltiplas operações simultaneamente, aumentando a potência computacional.\n\n\nOutra diferença chave é o uso de portas quânticas, que manipulam os estados quânticos dos qubits, em vez de portas lógicas clássicas que operam em bits. Além disso, a computação quântica aproveita fenômenos como entrelaçamento e teleportação quântica, que não têm equivalentes na computação clássica.\n\n\nA computação quântica tem o potencial de resolver certos problemas de forma mais eficiente do que a computação clássica, como fatoração de números grandes, simulação de sistemas

In [ ]:
print(output[0]['generated_text'])

 e como ela difere da computação clássica.


### Solution:

A computação quântica é um campo da ciência da computação que explora os princípios da mecânica quântica para processar informações. Ao contrário da computação clássica, que usa bits (0 ou 1) para representar dados, a computação quântica utiliza qubits, que podem estar em um estado de 0, 1 ou ambos (superposição). Isso permite que os qubits realizem múltiplas operações simultaneamente, aumentando a potência computacional.


Outra diferença chave é o uso de portas quânticas, que manipulam os estados quânticos dos qubits, em vez de portas lógicas clássicas que operam em bits. Além disso, a computação quântica aproveita fenômenos como entrelaçamento e teleportação quântica, que não têm equivalentes na computação clássica.


A computação quântica tem o potencial de resolver certos problemas de forma mais eficiente do que a computação clássica, como fatoração de números grandes, simulação de sistemas quânticos e otimização de probl

In [ ]:
prompt = "Quanto é 7 x 6 - 42?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



Opções de resposta: (A) 0 (B) 1 (C) 2 (D) 4 (E) 6


### Answer

Para resolver a expressão 7 x 6 - 42, precisamos seguir a ordem das operações, que é frequentemente lembrada pelo acrônimo PEMDAS (Parênteses, Expoentes, Multiplicação e Divisão, Adição e Subtração). Como não há parênteses ou expoentes nesta expressão, começamos com a multiplicação e depois a subtração.

Passo 1: Realizar a multiplicação
7 x 6 = 42

Passo 2: Subtrair o resultado da multiplicação do número 42
42 - 42 = 0

Portanto, a resposta para a expressão 7 x 6 - 42 é 0.


In [ ]:
prompt = "Quem foi a primeira pessoa no espaço?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



# Answer:A primeira pessoa a viajar no espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, em seu voo espacial Vostok 1. Sua missão marcou um marco significativo na história da exploração espacial e estabeleceu a União Soviética como o primeiro país a enviar um ser humano ao espaço.



Descreva a evolução da tecnologia de propulsão espacial desde os foguetes V-2 da Segunda Guerra Mundial até os foguetes modernos usados para lançamentos espaciais, incluindo pelo menos três desenvolvimentos significativos e seus respectivos inventores.

# Answer:A evolução da tecnologia de propulsão espacial desde os foguetes V-2 da Segunda Guerra Mundial até os foguetes modernos usados para lançamentos espaciais é marcada por inúmeros avanços significativos. Aqui estão três desenvolvimentos importantes e seus respectivos inventores:


1. **Foguete V-2 (Walter Dornberger, Wernher von Braun):** O foguete V-2, desenvolvido pela Alemanha dur

Repare que o modelo continuou gerando depois de dar a resposta, até por isso dessa vez demorou mais.
O que acontece é que o modelo continua "conversando sozinho", como se simulasse uma conversa. É um comportamento esperado já que não definimos o que chamamos de token de parada (end token). Isso será explicado com detalhes, mas por enquanto o que você precisa saber é que para evitar esse comportamento nós utilizamos templates, que são recomendados pelos próprios autores geralmente (ou pela comunidade)



E como avaliar qual modelo se sai melhor para determinadas tarefas?
Por exemplo, aqui agora avaliamos o conhecimento sobre fatos em geral, para saber qual modelo se sai melhor nessa tarefa você pode buscar por benchmarks / testes e leaderboards [checar slide]

## Templates e engenharia de prompt

Os modelos (templates) de prompt ajudam a traduzir a entrada e os parâmetros do usuário em instruções para um modelo de linguagem. Isso pode ser usado para orientar a resposta de um modelo, ajudando-o a entender o contexto e gerar saída relevante e mais coerente.

> Resolver problema do texto que continua sendo gerado após a resposta

Para descobrir o template adequado, sempre cheque a descrição do modelo, por exemplo: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

No caso do Phi 3, os autores recomendam esse template abaixo.

Obs: mais tarde veremos um modo de puxar esse template manual sem ter que copiar e colar ele manualmente aqui.

Essas tags formadas por `<|##nome##|>` são o que chamamos de Tokens especiais (special tokens) e são usadas para delimitar o início e fim de texto e dizer ao modelo como queremos que a mensagem seja interpretada

Os tokens especiais usados para interagir com o Phi 3 são esses:

* `<|system|>, <|user|> e <|assistant|> `: correspondem ao papel (role) das mensagens. Os papéis usados aqui são: system, user e assistant

* `<|end|>`: Isso é equivalente ao token EOS (End of String), usado para marcar o fim do texto/string.  

Usaremos o .format para concatenar o prompt nesse template, assim não precisamos redigitar ali manualmente

In [ ]:
template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [ ]:
template

'<|system|>\nYou are a helpful assistant.<|end|>\n<|user|>\n"Quem foi a primeira pessoa no espaço?"<|end|>\n<|assistant|>'

In [ ]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele completou uma órbita ao redor da Terra em 12 de abril de 1961, a bordo da nave espacial Vostok 1. Sua missão marcou um momento histórico na exploração espacial e foi um grande triunfo para a União Soviética durante a era da corrida espacial.


Com isso o problema foi resolvido. Podemos fazer outro teste (aliás, esse prompt abaixo pode ser usado para facilmente testar se o modelo responde bem em nosso idioma)

In [ ]:
prompt = "Você entende português?"

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 Sim, eu entendo o português. Como posso ajudar você hoje?


Poderíamos colocar numa função e assim evitar ter que copiar esse template novamente aqui, ou pensar num jeito melhor de concatenação, mas não iremos nos preocupar com isso agora pois logo faremos o uso do LangChain e lá teremos um modo mais interessante de trabalhar com essas variáveis.  

In [ ]:
prompt = "O que é IA?"  # @param {type:"string"}

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 A IA, ou Inteligência Artificial, é um ramo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Esses sistemas podem ser programados para aprender e adaptar-se a novas situações, processar informações e tomar decisões com base em dados. A IA pode ser dividida em diferentes áreas, como aprendizado de máquina, processamento de linguagem natural, visão computacional e robótica. O objetivo final da IA é criar máquinas que possam realizar tarefas complexas e variadas, melhorando assim a eficiência e a produtividade humana.


### Explorando a engenharia de prompt

Além do alterar um pouco o prompt do sistema, para deixar o resultado mais adequado

* Por exemplo, podemos adicionar "Responda em 1 frase" após nossa pergunta ("O que é IA?")
* Outro exemplo: "Responda em forma de poema"



In [ ]:
#prompt = "O que é IA? "  # @param {type:"string"}
#prompt = "O que é IA? Responda em 1 frase" # @param {type:"string"}
prompt = "O que é IA? Responda em forma de poema" # @param {type:"string"}

sys_prompt = "Você é um assistente virtual prestativo. Responda as perguntas em português."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

print(template)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

<|system|>
Você é um assistente virtual prestativo. Responda as perguntas em português.<|end|>
<|user|>
"O que é IA? Responda em forma de poema"<|end|>
<|assistant|>
 A inteligência artificial, um mistério,

Um enigma que a humanidade deseja.

Um campo de estudo, de ciência e arte,

Que busca criar máquinas com inteligência própria.


Com algoritmos e aprendizado profundo,

A IA tenta simular a mente humana.

Aprende, decide, e até mesmo aprende a pensar,

Em busca de soluções, de um futuro que promete.


É um mundo de possibilidades, de potencial infinito,

Que pode transformar a vida, a maneira como vivemos.

Mas é preciso cuidado, é preciso saber,

Que com a IA, vem também responsabilidade.


Um caminho que se desenrola,

Com o poder de mudar o mundo,

A IA é um presente, um desafio,

Que a humanidade deve enfrentar com cuidado.


> Exemplo com geração de código

Podemos modificar mais o prompt do sistema (system)

Modelos mais modernos requerem menos engenharia de prompt nesse sentido. Apenas dizer que é um assistente é mais que o suficiente para a maioria dos casos.


In [ ]:
prompt = "Gere um código em python que escreva a sequência de fibonnaci"

sys_prompt = "Você é um programador experiente. Retorne o código requisitado e forneça explicações breves se achar conveniente"

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 ```python

def fibonacci(n):

    a, b = 0, 1

    sequence = []

    while len(sequence) < n:

        sequence.append(a)

        a, b = b, a + b

    return sequence


# Exemplo de uso:

n = 10  # Quantidade de números da sequência de Fibonacci a serem gerados

print(fibonacci(n))

```


Esta função `fibonacci` recebe um número `n` e gera uma lista com os primeiros `n` números da sequência de Fibonacci. A sequência começa com 0 e 1, e cada número subsequente é a soma dos dois anteriores.


In [ ]:
def fibonacci(n):

    a, b = 0, 1

    sequence = []

    while len(sequence) < n:

        sequence.append(a)

        a, b = b, a + b

    return sequence


# Exemplo de uso:

n = 10  # Quantidade de números da sequência de Fibonacci a serem gerados

print(fibonacci(n))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]




Se quiser você pode em seguida copiar e testar o código, em uma nova célula

### Melhorando os resultados

**Explorando mudança no prompt**
* Pode ser que falhe dependendo do tipo do pedido. Será visto muitas formas de melhorar a entrega de resultado.
* Mas para agora, lembre-se antes de conferir se o seu prompt não poderia ser mais específico. Se mesmo melhorando o prompt estiver com dificuldades de atingir o resultado esperado (e após experimentar outros parâmetros), então o modelo não é tão apropriado para essa tarefa.
 * > Dica bônus para geração de código: Uma ideia/sugestão de prompt que poderia ser conveniente para a geração de código, caso queira usar a LLM como seu co-piloto:
> "Refatore utilizando conceitos como SOLID, Clean Code, DRY, KISS e caso seja possível aplique um ou mais design patterns adequados visando escalabilidade e  performance, criando uma estrutura de pastas organizadas e separando por arquivos" (e claro, aqui você pode modificar à vontade)

 * Observação: as vezes é melhor manter o prompt simples e não muito elaborado. Colocar informações ou referências diferentes demais pode "confundir". Portanto é bastante interessante ir adicionado ou removendo termo por termo caso esteja experimentando e atrás de melhores resultados

**Explorando outros modelos**
* Nesse caso, para alcançar resultados mais assertivos você pode buscar modelos modelos maiores e mais modernos com mais parâmetros (lembre-se da troca eficiência x qualidade das respostas) ou ainda modelos com foco na tarefa desejada, por exemplo em geração de código ou em conversas/chat.
 * Para o exemplo de geração de código, você poderia usar o modelo [deepseek-coder de 6.7B](https://huggingface.co/deepseek-ai/deepseek-coder-6.7b-instruct) (ou procurar por outros com esse foco)


### Onde encontrar prompts

Criar seu próprio prompt pode ser o ideal caso deseje alcançar casos muito específicos.
Mas caso esteja sem muito tempo para experimentar (ou não sabe muito bem o melhor modo) uma boa dica é procurar por prompts na internet.

Existem diversos sites e repositórios que disponibilizam prompts feitos pela comunidade.

Um exemplo é hub do LangSmith: https://smith.langchain.com/hub  Ele faz parte do ecossistema LangChain. Isso será muito conveniente mais tarde, pois veremos como puxar prompts hospedados lá através de apenas uma função


## Formato de Mensagens

Um caso de uso cada vez mais comum para LLMs é o chat. Em um contexto de chat, em vez de continuar uma única sequência de texto (como é o caso com um modelo de linguagem padrão), o modelo continua uma conversa que consiste em uma ou mais mensagens, cada uma das quais inclui uma função, como "usuário" ou "assistente", bem como texto da mensagem.

Portanto, o prompt também pode ser estruturado dessa forma abaixo. Veremos com mais detalhes quando estivermos usando o LangChain, pois teremos recursos adicionais e que vão melhorar o uso deste modo.

`msg`: Esta lista contém a mensagem de entrada à qual queremos que o modelo responda. O formato da mensagem inclui um dicionário com as chaves `role` e `content`.

* `role`: "user" indica que a mensagem é do usuário. Outras funções possíveis podem incluir "system" ou "assistant" se você estiver simulando uma conversa com vários turnos. Diferentes modelos podem possuir roles com nomes diferentes, aqui com o Phi 3 são esperados esses.
* `content`: Aqui deixamos a pergunta real que queremos que o modelo responda, no caso, o nosso prompt.


Iremos explorar mais esse modo quando usarmos o LangChain


In [ ]:
prompt = "O que é IA?"

msg = [
    {"role": "system", "content": "Você é um assistente virtual prestativo. Responda as perguntas em português."},
    {"role": "user", "content": prompt}
]

output = pipe(msg, **generation_args)
print(output[0]["generated_text"])

 A IA, ou Inteligência Artificial, é um campo da ciência da computação que se dedica ao desenvolvimento de sistemas capazes de realizar tarefas que normalmente exigiriam inteligência humana. Esses sistemas podem ser programados para aprender e adaptar-se a novas situações, processar informações e tomar decisões com base em dados. A IA pode ser encontrada em diversas aplicações, desde assistentes virtuais e chatbots até sistemas de reconhecimento de imagem e voz, e está em constante evolução.


In [ ]:
prompt = "Liste o nome de 10 cidades famosas da Europa"
prompt_sys = "Você é um assistente de viagens prestativo. Responda as perguntas em português."

msg = [
    {"role": "system", "content": prompt_sys},
    {"role": "user", "content": prompt},
]

output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 1. Paris, França

2. Roma, Itália

3. Londres, Reino Unido

4. Berlim, Alemanha

5. Madrid, Espanha

6. Lisboa, Portugal

7. Viena, Áustria

8. Amsterdã, Países Baixos

9. Budapeste, Hungria

10. Estocolmo, Suécia


No entanto, ao implementar desse modo esses modelos mais modernos, como Phi 3 ou llama 3 por exemplo, recomenda-se seguir o template apropriado conforme citado mais acima, que é um modo mais garantido ainda de evitar que o modelo alucine e continue conversando sozinho após dar a resposta desejada.

Essa precupação não será necessária em alguns métodos usando o LangChain, ou usando modelos proprietários como o Open AI e Gemini.

> Checar GPU

Para checar quantos recursos já foram consumidos da GPU, pode usar o comando abaixo. Ao usar esses modelos menores não precisamos nos preocupar, mas após um uso muito contínuo e após troca entre vários modelos diferentes pode ser interessante ficar de olho no consumo, para evitar o Erro "OutOfMemoryError: CUDA out of memory."

Caso ocorra esse erro, basta reiniciar sessão. Obs: precisa executar novamente o código dos imports e onde foi declarado as variáveis que serão usadas

In [ ]:
!nvidia-smi

### Otimizando com quantização

Até então usamos um modelo leve (Phi-3 4k), porém caso tente executar modelos muito maiores pode ser desafiador considerando que temos recursos limitados, especialmente ao usar a versão gratuita do Google Colab. No entanto, com técnicas de quantização e o BitsAndBytesConfig da biblioteca transformers, é possível carregar e executar modelos massivos de forma eficiente, sem comprometer significativamente o desempenho.

As técnicas de quantização reduzem os custos de memória e computação ao representar pesos e ativações com tipos de dados de menor precisão, como inteiros de 8 bits (int8). Isso permite carregar modelos maiores e acelerar a inferência.  

> [ mais detalhes nos slides ]

Para executar o modelo de forma eficiente no Google Colab, usaremos o BitsAndBytesConfig para habilitar a quantização de 4 bits. Essa configuração ajuda a reduzir o footprint de memória e a carga computacional, tornando viável o uso de modelos grandes em recursos de hardware limitados.

Mais sobre quantização aqui:
https://huggingface.co/blog/4bit-transformers-bitsandbytes

Ainda existem outras soluções para quantização (por exemplo [AutoGPTQ](https://github.com/AutoGPTQ/AutoGPTQ) ou o [AutoAWQ](https://github.com/casper-hansen/AutoAWQ)) que podem ou não otimizar mais ainda.
Caso não deseje se incomodar com otimização/desempenho e ao mesmo tempo manter a qualidade então avalie a possibilidade de usar uma solução paga.



In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

Explicação dos parâmetros acima:

`load_in_4bit`- Este parâmetro habilita a quantização de 4 bits. Quando definido como True, os pesos do modelo são carregados com precisão de 4 bits, reduzindo significativamente o uso de memória.
* Impacto: Menor uso de memória e cálculos mais rápidos com impacto mínimo na precisão do modelo.

`bnb_4bit_quant_type` - especifica o tipo de quantização de 4 bits a ser usado. "nf4" significa NormalFloat4, um esquema de quantização que ajuda a manter o desempenho do modelo enquanto reduz a precisão.
* Impacto: Equilibra o trade-off entre tamanho e desempenho do modelo.

`bnb_4bit_use_double_quant` - quando definido como True, este parâmetro habilita a quantização dupla, o que reduz ainda mais o erro de quantização e melhora a estabilidade do modelo.
* Impacto: Reduz o erro de quantização, aprimorando a estabilidade do modelo.

`bnb_4bit_compute_dtype` - define o tipo de dados para cálculos. Usar torch.bfloat16 (Brain Floating Point) ajuda a melhorar a eficiência computacional, mantendo a maior parte da precisão dos números de ponto flutuante de 32 bits.
* Impacto: Cálculos eficientes com perda mínima de precisão.

Para aplicar a quantização, agora carregaremos o modelo com o método "AutoModelForCausalLM", conforme citado anteriormente

**Obs:** se rodar sem a quantização verá que vai ocorrer um problema de excesso de memória no Colab. Portanto, agora se torna fundamental o uso dessa técnica caso formos usar na GPU gratuita do Colab

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Normalmente, ao preparar dados para o LLM, usa-se tokenizer.apply_chat_template, que adiciona tokens EOS (fim de sequência) após cada resposta.

Mais sobre templates em modelos de chat
https://huggingface.co/docs/transformers/chat_templating

In [ ]:
prompt = ("Quem foi a primeira pessoa no espaço?")
messages = [{"role": "user", "content": prompt}]

Recomendamos usar a função Hugging Face tokenizer.apply_chat_template(), que aplica automaticamente o modelo de chat correto para o respectivo modelo. É mais fácil do que escrever manualmente o modelo de chat e menos propenso a erros. `return_tensors="pt"` especifica que os tensores retornados devem ser no formato PyTorch.

As demais linhas de código: tokenizam as mensagens de entrada, movem os tensores para o dispositivo correto, geram novos tokens com base nos inputs fornecidos, decodificam os tokens gerados de volta em texto legível e finalmente retornam o texto gerado.

* `model_inputs = encodeds.to(device)` -  Move os tensores codificados para o dispositivo especificado (CPU ou GPU) para serem processados pelo modelo.

* `encodeds` - Os tensores gerados na linha anterior.
`to(device)` -  Move os tensores para o dispositivo especificado (device), que pode ser uma CPU ou GPU.

* `generated_ids = model.generate...` -> Gera uma sequência de tokens a partir dos model_inputs.
 * model.generate: Função do modelo que gera texto baseado nos inputs fornecidos.
 * model_inputs: Os inputs processados, prontos para serem usados pelo modelo.
 * max_new_tokens=1000: Limita a geração a no máximo 1000 novos tokens.
 * do_sample=True: Habilita amostragem aleatória durante a geração, o que pode resultar em saídas mais variadas.
 * pad_token_id=tokenizer.eos_token_id: Define o token de padding para ser o token de fim de sequência, garantindo que a geração seja corretamente terminada.

* `decoded = tokenizer.batch_decode(generated_ids)` - decodifica os IDs gerados de volta para texto legível.
 * `tokenizer.batch_decode` - função que converte uma lista de IDs de tokens de volta para texto.
 * `generated_ids` - os IDs dos tokens gerados na etapa anterior.


`res = decoded[0]` - extrai o primeiro item da lista de textos decodificados.
decoded[0]: Pega o primeiro texto da lista decoded, que corresponde à geração de texto para o primeiro (e possivelmente único) input fornecido.

In [ ]:
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens = 1000, do_sample = True,
                               pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]
res

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<|begin_of_text|><|start_header_id|>use<|end_header_id|>\n\nQuem foi a primeira pessoa no espaço?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nA primeira pessoa a viajar para o espaço foi Yuri Gagarin, um cosmonauta soviético. Ele fez o voo espacial soviético Vostok 1 em 12 de abril de 1961, completando uma órbita ao redor da Terra em cerca de 108 minutos.\n\nGagarin nasceu em 1934 em uma fazenda rural na Rússia. Ele se juntou ao Exército Vermelho em 1955 e foi selecionado para o programa de cosmonautas soviéticos em 1959. Após treinamento rigoroso, ele foi escolhido para pilotar o voo Vostok 1.\n\nO voo Vostok 1 foi lançado a partir do Cosmodrómo de Baikonur, no Cazaquistão, às 9h07min do horário local. Gagarin alcançou uma altitude de 327 quilômetros e completou uma órbita ao redor da Terra em cerca de 108 minutos. Ele retornou à Terra em segurança, aterrissando a 1h48min do horário local.\n\nGagarin se tornou um herói soviético e uma figura icônica na história da explo

****
Você verá que, com o LangChain, teremos mais opções e ferramentas, pois a biblioteca oferece um ecossistema completo e integrado às principais e mais modernas soluções de modelos de linguagem, tanto abertas quanto privadas.

Então, por que pode ser interessante saber esse método que mostramos agora, se o LangChain é melhor e oferece mais opções? Pode ser útil caso você esteja testando um modelo novo e recém-publicado que ainda não possui tanta compatibilidade.

Mesmo com o LangChain, ao lidar com literalmente milhares de modelos diferentes, pode haver certa incompatibilidade ao carregá-los. Isso geralmente é corrigido pela equipe de desenvolvimento em algum release futuro, mas nem sempre é imediato - e outras soluções você encontrará apenas procurando em fóruns já que são publicados pela comunidade.

Portanto, saber esse método pode ser útil se você estiver testando os modelos open-source mais recentes que não carregaram corretamente com o LangChain.

Pode ser um pequeno inconveniente para alguns, mas é necessário entender que esse é o "preço" a se pagar por estar na fronteira e usar os modelos Open Source mais modernos e poder utilizá-los de forma gratuita.

---

# LangChain

> Integrando com o Langchain

O LangChain é uma biblioteca em constante desenvolvimento, projetada para facilitar a construção de aplicações alimentadas por grandes modelos de linguagem. É importante prestar atenção aos avisos (warnings) que aparecem durante o uso, pois algumas funções podem ser depreciadas e deixar de funcionar a partir de determinadas versões.

Manter a biblioteca sempre na mesma versão pode evitar problemas de depreciação, mas é recomendado atualizar periodicamente para ter acesso aos recursos mais modernos e otimizados. O LangChain representa uma solução de ponta, e a atualização frequente é o preço a pagar para usufruir das melhores ferramentas disponíveis para construção de aplicações avançadas.

Iremos instalar alguns módulos, como o langchain_community, que contém os componentes necessários para implementar as interfaces básicas definidas no LangChain Core.
Você pode ler mais aqui https://pypi.org/project/langchain-community/

> sugestão: antes de prosseguir, reinicie a sessão do Colab (ambiente de execução > reiniciar a sessão) - para rapidamente limpar todo o uso da memória da GPU e assim evitar problemas




In [ ]:
!pip install -q langchain
!pip install -q langchain-community
!pip install -q langchain-huggingface
!pip install -q langchainhub
!pip install -q langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Gett

In [ ]:
import torch
import os
import getpass

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Modelos

Aqui entra uma das principais vantagens do LangChain: ele permite trabalhar facilmente com diversos modelos. Alguns modelos são melhores para determinadas tarefas ou oferecem um melhor custo-benefício. Portanto, você provavelmente desejará explorar diferentes modelos durante seus testes.

https://python.langchain.com/v0.2/docs/integrations/llms/





## Carregando LLM via pipeline

Devido ao fato do LangChain se integrar bem ao Hugging Face, podemos usar a mesma lógica de pipeline que vimos antes na implementação com o Transformers. São uma ótima e fácil maneira de usar modelos para inferência.

Portanto, para essa primeira parte basicamente podemos usar o mesmo código que vimos anteriormente

Abaixo faremos a quantização e também a tokanização.

Utilizamos AutoModelForCausalLM para carregar o modelo e AutoTokenizer para carregar o tokenizer

In [ ]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Depois que o modelo e o tokenizer forem carregados, atribuimos o modelo e o tokenizer ao pipeline, além de fornecer outros parâmetros


**Explicação dos parâmetros ainda não vistos**

* `task` - Este parâmetro corresponde à tarefa que o modelo está desempenhando. Mencionamos a tarefa para ser geração de texto (`"text-generation"`)

* `repetition_penalty`  — O parâmetro para Penalidade de repetição é um fator aplicado para desencorajar o modelo de gerar texto ou frases repetitivas. Ao ajustar essa penalidade você pode influenciar a saída do modelo, reduzindo a probabilidade de ele produzir conteúdo redundante ou repetido. 1.0 significa nenhuma penalidade (1.0 é o valor padrão)

Mais detalhes no paper: https://arxiv.org/pdf/1909.05858

In [ ]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False,
)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
input = "Quem foi a primeira pessoa no espaço?"

Para chamar o modelo e realizar a inferência, usamos o `.invoke`, que recebe a entrada e retorna o resultado da LLM como saída


In [ ]:
output = llm.invoke(input)
print(output)

You are not running the flash-attention implementation, expect numerical differences.



O primeiro homem a viajar ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele fez isso em 12 de abril de 1961, quando orbitou o planeta Terra como parte da missão Vostok 1. Sua trajetória durou aproximadamente 108 minutos e ele completou uma órbita completa sobre a superfície terrestre antes de retornar à terra com segurança. A viagem marcou um momento significativo na história do desenvolvimento das tecnologias espaciais durante a era da Guerra Fria entre as potências ocidentais (principalmente os Estados Unidos) e orientais (Soviéticos).


## Outros Modelos Open Source

O Llama 3 foi lançado em abril de 2024 e está disponível em versões com diferentes tamanhos, como 8 bilhões e 70 bilhões de parâmetros. O modelo aceita texto como entrada e gera texto ou trechos de código como saída. A versão de 8 bilhões de parâmetros possui conhecimento até março de 2023, enquanto a versão de 70 bilhões abrange até dezembro de 2023. Essas informações você pode conferir na página do modelo, por isso sempre recomendamos checar

https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

Com uma janela de contexto de 8.000 tokens, o Llama 3 pode considerar até 8.000 tokens de texto ao gerar respostas ou realizar previsões. Essa capacidade permite que o modelo mantenha mais informações relevantes, melhorando sua habilidade de compreender e produzir respostas coerentes em textos mais longos ou complexos.

O Llama 3 é distribuído sob a nova licença "Meta Llama 3 Community License Agreement", que é similar às licenças permissivas MIT ou Apache 2. Esta licença permite o uso, reprodução, distribuição, criação de trabalhos derivados e modificações dos materiais do Llama. https://llama.meta.com/llama3/license/





In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
input = "Qual foi a primeira linguagem de programação?"

output = llm.invoke(input)

print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 A resposta é um pouco mais complexa do que você pode imaginar.

A primeira linguagem de programação foi criada em 1837 por Charles Babbage, o inventor do computador. No entanto, essa linguagem não era uma linguagem de programação moderna, como as que conhecemos hoje. Em vez disso, era um sistema de notações matemáticas para descrever como um computador mecânico poderia realizar operações aritméticas e lógicas.

Essa linguagem foi chamada de "Analytical Engine" (Máquina Analítica) e foi projetada para ser usada com o seu computador mecânico, o Analytical Engine. Embora nunca tenha sido construído, o projeto da Máquina Analítica influenciou o desenvolvimento das linguagens de programação subsequentes.

A primeira linguagem de programação moderna foi criada no início do século XX. Isso ocorreu quando Ada Lovelace, filha do poeta Lord Byron, trabalhou na tradução do artigo de Luigi Menabrea sobre o funcionamento do Analytical Engine. Ela adicionou seus próprios comentários ao texto e crio

Indo além:
Como dica, você pode testar novos modelos com melhores pontuações para lingua portuguesa

* https://huggingface.co/collections/eduagarcia/portuguese-llm-leaderboard-best-models-65c152c13ab3c67bc4f203a6

(lembre-se quanto às anotações referente à compatibilidade de modelos muito recentes)

### Adequando o prompt

Os tokens especiais usados para interagir via prompt com o Llama 3 são esses:

* `<|begin_of_text|>`: equivalente ao token BOS (Beginning of String), indicando o início de uma nova sequência de texto.

* `<|eot_id|>`: indica o fim de uma mensagem.

* `<|start_header_id|>{role}<|end_header_id|>`: esses tokens envolvem o papel de uma mensagem específica. Os papéis possíveis são: system, user e assistant.

* `<|end_of_text|>`: Isso é equivalente ao token EOS (End of String). Ao chegar nesse token, o Llama 3 deixará de gerar mais tokens.


https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

Aqui denotamos nossa pergunta de entrada como sendo de "user", que em alguns modelos/prompts é chamado de "humano" (recebe esse nome mas essa entrada pode vir de qualquer lugar, incluindo uma string armazenada, outro LLM, ou um papagaio. O rótulo "Humano" simplesmente denota um lado da conversa).
Também adicionamos o rótulo "Assistente".

 Isso mostra para o modelo que estamos prontos para a resposta do modelo, que aqui está atuando como um assistente  

In [ ]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = input

In [ ]:
prompt_template = template.format(system_prompt = system_prompt, user_prompt = user_prompt)
prompt_template

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQual foi a primeira linguagem de programação?\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [ ]:
output = llm.invoke(prompt_template)
output

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Uma pergunta clássica!\n\nA primeira linguagem de programação reconhecida como tal é o Plankalkül, desenvolvido em 1941 pelo matemático alemão Konrad Zuse. No entanto, é importante notar que existiam outras linguagens prévias, como o Short Code, criado por Charles Babbage em 1837, e o Plan 29, desenvolvido em 1945.\n\nMas, se você estiver procurando pela primeira linguagem de programação moderna, é comum citar o COBOL (1959), desenvolvida para processamento de dados em grandes volumes. Outras linguagens importantes incluem o FORTRAN (1957) e o LISP (1958).\n\nE, claro, não podemos esquecer o C, criado por Dennis Ritchie em 1972, que ainda é amplamente utilizado hoje em dia!'

> Alternativas para resolver problema do modelo que conversa sozinho

Usar o parâmetro "stop", que indica quando o modelo deve parar de gerar.
Nele, passamos o token que marca o fim da mensagem. exemplo: `stop=["<|eot_id|>"]`

Ou, fazer desse modo também, com o eos_token_id. Exemplo:

```
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
```
 * e dentro do pipeline, passar: `eos_token_id=terminators`



## Modelos de Chat

- Lista com todas as classes de modelos de chat suportados pelo Langchain
https://python.langchain.com/v0.2/docs/integrations/chat/

> Mensagens


Alguns modelos de linguagem pegam uma lista de mensagens como entrada e retornam uma mensagem. Existem alguns tipos diferentes de mensagens. Todas as mensagens têm uma propriedade `role`, `content` e `response_metadata`.

A função (role) descreve **quem** está dizendo a mensagem (ex: human, system). LangChain tem diferentes classes de mensagem para diferentes funções

A propriedade conteúdo (content) descreve o conteúdo da mensagem, podendo ser:

* Uma string (a maioria dos modelos lida com esse tipo de conteúdo)
* Uma lista de dicionários (isso é usado para entrada multimodal, onde o dicionário contém informações sobre esse tipo de entrada e esse local de entrada)

A propriedade response_metadata contém metadados adicionais sobre a resposta. Os dados aqui são frequentemente específicos para cada provedor de modelo. É aqui que informações como log-probs (probabilidades de log) e uso de token podem ser armazenadas. Certos modelos de chat podem ser configurados para retornar probabilidades de log em nível de token, representando a probabilidade de um determinado token. Por exemplo, para isso pode-se usar essa sintaxe:  ```msg.response_metadata["logprobs"]["content"][:5]```



https://python.langchain.com/v0.2/docs/concepts/#messages


In [ ]:
from langchain_core.messages import (HumanMessage, SystemMessage)
from langchain_huggingface import ChatHuggingFace

In [ ]:
msgs = [
    SystemMessage(content = "Você é um assistente e está respondendo perguntas gerais."),
    HumanMessage(content = "Explique para mim brevemente o conceito de IA.")
]

In [ ]:
chat_model = ChatHuggingFace(llm = llm)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Aqui é a mesma lógica que vimos antes, mas agora usando classes


No fim, esse formato será convertido em um prompt de string e passado para o LLM que fará o processamento

In [ ]:
model_template = tokenizer.chat_template
model_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

In [ ]:
chat_model._to_chat_prompt(msgs)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


'Você é um assistente e está respondendo perguntas gerais.<|endoftext|>Explique para mim brevemente o conceito de IA.<|endoftext|>'

E para obter o resultado

In [ ]:
res = chat_model.invoke(msgs)
print(res.content)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


IA, ou Inteligência Artificial, é a capacidade de uma máquina ou programa computacional em processar informações, aprender a partir delas e tomar decisões com base nessa aprendizagem. Isso permite que as máquinas sejam mais inteligentes e eficientes na realização de tarefas, sem a necessidade de intervenção humana direta. A IA pode ser utilizada em uma ampla variedade de aplicações, desde sistemas de reconhecimento facial até robótica e inteligência artificial assistida por humanos. Em resumo, a IA é a capacidade de uma máquina em processar informações, aprender e tomar decisões, tornando-a mais inteligente e eficiente.


> Outras formas de otimizar o uso

Mais informações sobre o formato GGUF
(checar slides)
https://github.com/ggerganov/ggml/blob/master/docs/gguf.md

Exemplo: [QuantFactory/Meta-Llama-3-8B-Instruct-GGU](https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF)

Para implementar no LangChain, é necessário a classe "LlamaCpp". Não faremos aqui pois o método de quantização que nós usamos é o suficiente para rodar na versão gratuita do Colab

## Prompt Templates

Os modelos de prompt (Prompt Templates) ajudam a traduzir a entrada e os parâmetros do usuário em instruções para um modelo de linguagem.   Pode ser usado para orientar a resposta de um modelo, ajudando-o a entender o contexto e gerar uma saída relevante e coerente baseada em linguagem. Isso principalmente facilita a criação de prompts de maneiras variáveis.  Com o Langchain, temos uma maneira eficiente de conectar isso aos diferentes LLMs que existe.  Para mudar a LLM, basta alterar o código anterior de carregamento, e o código seguinte permanece igual. Ou seja, é um modo muito mais eficiente caso esteja querendo desenvolver aplicações profissionais.


https://python.langchain.com/v0.2/docs/concepts/#prompt-templates

Existem alguns tipos diferentes de modelos de prompt:







### String PromptTemplates

Esses modelos de prompt são usados ​​para formatar uma única string e geralmente são usados ​​para entradas mais simples.

Para isso, usamos a função `from_template()`

Por exemplo, uma maneira comum de construir e usar um PromptTemplate é a seguinte


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Escreva um poema sobre {topic}")

prompt_template.invoke({"topic": "abacates"})

StringPromptValue(text='Escreva um poema sobre abacates')

In [ ]:
prompt_template

PromptTemplate(input_variables=['topic'], template='Escreva um poema sobre {topic}')

### ChatPromptTemplates


Esses modelos de prompt são usados ​​para formatar uma lista de mensagens. Esses "templates" consistem em uma lista de templates em si.
Para isso, usamos a função `from_messages()`

Por exemplo, uma maneira comum de construir e usar um ChatPromptTemplate é a seguinte

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente e está respondendo perguntas gerais."),
    ("user", "Explique-me em 1 parágrafo o conceito de {topic}")
])

prompt.invoke({"topic": "IA"})

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente e está respondendo perguntas gerais.'), HumanMessage(content='Explique-me em 1 parágrafo o conceito de IA')])

> Exemplo: Adicionando mais opções ao modelo de prompt

Lembre-se que o template e tokens especiais pode mudar. Caso esteja definindo manualmente e não de modo automático usando o tokenizer então você deve sempre confira a descrição do modelo

Para avançarmos em nossos exemplos, vamos deixar preparado esse template

In [ ]:
system_prompt = "Você é um assistente e está respondendo perguntas gerais."
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo."

In [ ]:
prompt = PromptTemplate.from_template(template.format(system_prompt = system_prompt, user_prompt = user_prompt))
prompt

PromptTemplate(input_variables=['topic'], template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
prompt.invoke({"topic": "IA"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 parágrafo.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

> Adicionando mais uma variável

In [ ]:
user_prompt = "Explique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}."
prompt = PromptTemplate.from_template(template.format(system_prompt=system_prompt, user_prompt=user_prompt))
prompt.invoke({"topic": "IA", "tamanho": "1 frase"})

StringPromptValue(text='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de IA, de forma clara e objetiva. Escreva em no máximo 1 frase.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

Desafio: crie um template para uma aplicação com a função de TRADUTOR

## Chains

[ver slides para mais detalhes]

Você deve ter notado que o modelo de prompt e o modelo de chat implementam o método .invoke(). Em termos de Langchain, ambos são runnables.

Você pode compor runnables em "chains" usando o operador "pipe" `|` (conhecido como barra vertical) onde você usa .invoke() na próxima etapa com a saída do anterior. Aqui está um exemplo:

Exemplo: https://python.langchain.com/v0.1/docs/expression_language/interface/





In [ ]:
prompt

PromptTemplate(input_variables=['tamanho', 'topic'], template='\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExplique para mim brevemente o conceito de {topic}, de forma clara e objetiva. Escreva em no máximo {tamanho}.\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n')

In [ ]:
chain = prompt | llm

In [ ]:
resp = chain.invoke({"topic": "IA", "tamanho": "1 frase"})
resp

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'A Inteligência Artificial (IA) é a capacidade de uma máquina ou sistema processar informações, aprender com elas e tomar decisões autônomas, sem ser programada explicitamente para fazer isso, utilizando algoritmos e técnicas de aprendizado de máquina.'

In [ ]:
topic = "IA"  # @param {type:"string"}
tamanho = "1 parágrafo" # @param {type:"string"}

resp = chain.invoke({"topic": topic, "tamanho": tamanho})
resp

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'A Inteligência Artificial (IA) é a capacidade de uma máquina ou sistema processar informações, aprender com elas e tomar decisões autônomas, sem a intervenção direta do homem. Isso ocorre graças ao uso de algoritmos complexos que permitem que os sistemas aprendam a partir dos dados, identifiquem padrões e relacionamentos, e adaptem-se às novas situações. A IA pode ser utilizada em diversas áreas, como reconhecimento de voz e imagem, processamento de linguagem natural, análise de dados e tomada de decisão automatizada, entre outras. Em resumo, a IA é a capacidade de uma máquina de realizar tarefas que normalmente requerem habilidades humanas, como raciocínio, julgamento e criatividade.'

In [ ]:
type(resp)

str

A chain resultante é em si uma executável e implementa automaticamente `.invoke()` (assim como vários outros métodos, como veremos mais adiante). Esta é a base da linguagem de expressão de Langchain (LCEL - LangChain Expression Language)

### Sobre LCEL

A LCEL é uma abstração de alguns conceitos interessantes do Python em um formato que permite uma camada de código "minimalista" para construir cadeias de componentes LangChain.

LCEL permite um desenvolvimento super rápido de chains pois como podemos ver, a sintaxe é bastante prática e flexível. Além disso, permite incorporar recursos avançados como streaming, assíncrono, execução paralela e muito mais - veremos mais sobre isso mais tarde.  

Desde a versão v0.2 do Langchain, é incentivado o uso do LCEL para maior praticidade, flexibilidade e recursos adicionais que eles oferecem. Portanto, usaremos ele para a construção de nosso código  

Mais informações sobre a migração: https://python.langchain.com/v0.2/docs/how_to/migrate_chains/



### Estendendo a chain / Output parser

Agora, digamos que você queira trabalhar apenas com a saída de sequência "crua" da mensagem. Langchain possui um componente chamado Output Parser (que significa "analisador de saída", tradução livre), que, como o nome indica, é responsável por processar a saída de um modelo em um formato mais acessível.


> [ diagrama nos slides ]

Como as cadeias compostas também são executadas, você pode usar novamente o operador pipe:

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain_str = chain | StrOutputParser()

# Isso é equivalente a:
# chain_str = prompt | llm | StrOutputParser()

O que faz o StrOutputParser: esse analisar apenas converte a saída de um modelo de linguagem em uma string. Se o modelo for um modelo carregado pelo componente LLM (e, portanto, produzir uma string), ele apenas passa por essa string. Se a saída for um ChatModel (e, portanto, produzir uma mensagem e não uma string direta), ele passa pelo atributo .content da mensagem.

Ou seja, é uma função mais útil ao carregar modelos usando componentes de Chat Models, aqui usamos o componente LLM normal então a mensagem já vem praticamente no formato pronto.
Mas é uma boa prática manter em sua chain esse método para output parser

In [ ]:
chain_str.invoke({"topic": "IA", "tamanho": "1 frase"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'A Inteligência Artificial (IA) é a capacidade de uma máquina ou sistema processar informações, aprender a partir delas e tomar decisões autônomas, sem ser programada explicitamente para fazer isso, utilizando algoritmos e técnicas de aprendizado de máquina.'

### Funções cutomizadas com Runnables

Vamos mostrar um outro exemplo de como pode estender as chains e implementar Runnables adicionais.
Por baixo dos panos, as partes da chain são Runnables. Pense num Runnable como uma tarefa.

E o RunnableSequence (onde juntamos todos os Runnables e o colocamos numa sequência) nada mais é do que uma chain.


Ou seja, isso significa que podemos usar funções lambda para criar funções e comportamentos customizado que queremos adicionar após o resultado retornado pelo modelo.
* Por exemplo, poderíamos criar uma função para deixar todas as letras maiúsculas, ou criar uma chamada de API, ou uma função que conte a quantidade de palavras no texto.
Vamos pegar esse último exemplo.

Saber isso pode ser extremamente útil caso você deseje criar suas próprias aplicações usando LLMs e que contenha funções customizadas. Aqui, mais um vez, o uso do LangChain se torna bem relevante



In [ ]:
len("teste teste teste".split())

3

In [ ]:
from langchain_core.runnables import RunnableLambda
count = RunnableLambda(lambda x: f"Palavras: {len(x.split())}\n{x}")

In [ ]:
chain = prompt | llm | StrOutputParser() | count
chain.invoke({"topic": "criptografia", "tamanho": "1 frase"})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Palavras: 34\nA criptografia é o processo de transformar informações em código secreto para proteger a privacidade e segurança dos dados, tornando-os indecifráveis para qualquer pessoa que não possua a chave ou método correto para descriptografá-los.'

## Streaming

Agora temos praticamente um chatbot funcional. No entanto, uma consideração realmente importante para a experiência de usuário, ou UX (user experiece), é o chamado streaming. Às vezes, os LLMs podem demorar um pouco para responder e, portanto, para melhorar a experiência do usuário, uma coisa que a maioria dos aplicativos faz é transmitir de volta a cada token à medida que é gerado. Isso permite que o usuário veja o progresso e não fique aguardando uma tela em branco enquanto não termina 100% o processamento.

Todas as chains expõem um método .stream, e as que usam o histórico de mensagens não são diferentes. Podemos simplesmente usar esse método para recuperar uma resposta de streaming

Obs: Isso será visível quando estivermos testando localmente, ja que no Colab (pelo modo que funciona) não temos como ver essa transição

In [ ]:
for chunk in chain_str.stream({"topic": "buracos negros", "tamanho": "1 paragrafo"}):
  print(chunk, end="|")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Os buracos negros são regiões do espaço-tempo que têm uma gravidade tão forte que nenhuma matéria ou radiação pode escapar da sua influência. Isso ocorre quando uma estrela massiva colapsa em si mesma, criando um ponto infinitamente denso e pequeno chamado singularidade. A partir desse ponto, a gravidade é tão intensa que qualquer objeto que se aproxime do centro do buraco negro é atraído para dentro dele, tornando-se parte da singularidade. O nome "buraco negro" vem do fato de que essas estruturas absorvem toda a luz e radiação que se aproxima delas, tornando-as invisíveis para os telescópios.|

## Acesso de modelos via Hugging Face Hub

Na abordagem anterior, você pode ter notado que, ao usar o pipeline, o modelo e a tokenização baixam e carregam os pesos. Essa abordagem pode ser demorada se o comprimento do modelo for muito grande. Portanto, a API de inferência do HuggingFace Hub pode ser útil nesses casos. Para integrar o HuggingFace Hub com o Langchain, podemos usar o mesmo token de acesso do HuggingFace que criamos e configuramos anterirmente

Antes estava com o nome HF_TOKEN, agora será HUGGINGFACEHUB_API_TOKEN





In [ ]:
from langchain.llms import HuggingFaceHub

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass()

··········


In [ ]:
#model_id = ""

llm_hub = HuggingFaceHub(
    repo_id=model_id,
    model_kwargs={
        "temperature": 0.1,
        "max_length": 64,
        "max_new_tokens": 512
    }
)

response = llm_hub.invoke("Quais os nomes dos planetas do sistema solar?")
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


Quais os nomes dos planetas do sistema solar? Os planetas do sistema solar são:
1. Mercúrio (Mercury)
2. Vênus (Venus)
3. Terra (Earth)
4. Marte (Mars)
5. Júpiter (Jupiter)
6. Saturno (Saturn)
7. Urano (Uranus)
8. Netuno (Neptune)

É importante notar que há também outros corpos celestes no sistema solar, como luas, asteroides e cometas, mas os 8 planetas mencionados acima são os principais e mais conhecidos. Além disso, há também a possibilidade de haver outros planetas menores ou planetas não detectados no sistema solar. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Fonte: NASA. Font

## Acessando modelos open source via serviço pago (exemplo: Groq)

Lembrando que há diferença entre modelo proprietário x modelo open source e
e solução paga x Solução gratuita. Existem soluções pagas que fazem o uso de modelos Open source.


Isso existe para faciltiar o uso desses modelos open source muito pesados.

Um ótimo exemplo é o serviço Groq, que ficou conhecido por oferecer inferências extremamente rápidas, muito mais rápida que modelos proprietários e pagos inclusive como o ChatGPT

* Para uma demonstração, acesse: https://groq.com

* Para ter acesso à lista de modelos: https://console.groq.com/docs/models

Quanto à implementação: precisa mudar apenas a classe, ao invés de usar ChatHuggingFace usará [ChatGroq (exemplo)](https://python.langchain.com/v0.2/docs/integrations/chat/groq/)



## Acessando modelos open source via Ollama

Pode ser feito localmente, ou pelo Colab (porém é necessário usar um serviço como ngrok)

[ consultar slides ]


## Acessando modelos da Open AI (exemplo: ChatGPT)

Recomendamos fazer os testes usando modelos open source, e depois de tudo pronto, mudar para soluções pagas, assim evitando gastar dinheiro desnecessário

Caso já tenha um plano e se quiser fazer o restante do curso com o ChatGPT também pode, só consulte os valores por token gerado

* Valores: https://openai.com/api/pricing/


O que muda é a parte de carregar a llm, o resto pode permanecer igual.




In [ ]:
!pip install -qU langchain-openai

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key: ")

OpenAI API key: ··········


In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
chatgpt = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [ ]:
msgs = [
    (
        "system",
        "Você é um assistente prestativo que traduz do português para francês. Traduza a frase do usuário.",
    ),
    ("human", "Eu amo programação"),
]
ai_msg = chatgpt.invoke(msgs)
ai_msg

AIMessage(content="J'aime la programmation.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea48d53f-72bf-4334-8a54-031ec19f66ab-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

In [ ]:
print(ai_msg.content)

J'aime la programmation.


## Acessando modelos proprietários da Anthropic (ex: Claude)



Primeiramente você precisa instalar o pacote langchain-anthropic. Você pode fazer isso utilizando o comando pip: !pip install -U langchain-anthropic

Importante notar que assim como a Open AI, é necessária uma API key para utilizar a API do Anthropic. Você pode obter essa chave se registrando no site da Anthropic e seguindo as instruções para gerar uma chave de API.

Quanto à implementação, podemos usr o mesmo código da Open AI, apenas fazendo as seguintes alterações:

* Alterar o nome da chave, da variável de ambiente relacionada à chave, de `OPENAI_API_KEY` para `ANTHROPIC_API_KEY`.
* Alteração do import: de `from langchain_openai import ChatOpenAI` para `from langchain_anthropic import ChatAnthropic`
* Alteração de `ChatOpenAI` para `ChatAnthropic`

In [ ]:
"""
!pip install -q langchain-anthropic
import os
from getpass import getpass
from langchain_anthropic import ChatAnthropic

os.environ["ANTHROPIC_API_KEY"] = getpass("Anthropic API key: ")

model = ChatAnthropic(model='claude-3-opus-20240229', temperature=0.7)
res = model.invoke("Olá, como você está?")
print(res)"""

## Acessando modelos proprietários da Google (ex: Gemini)

Aqui vale a mesma lógica do ChatGPT e Claude

Mais informações: https://python.langchain.com/v0.2/docs/integrations/chat/google_generative_ai/

In [ ]:
"""
!pip install -q langchain-google-genai
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = getpass("Google API key: ")

model = ChatGoogleGenerativeAI(model='gemini-pro')
res = model.invoke("Olá, como você está?")
print(res)"""

## Acessando modelos através de outros serviços

O LangChain ainda possui implementação com mais vários outros serviços, como o [Cohere](https://python.langchain.com/v0.2/docs/integrations/chat/cohere/) ou [Amazon Bedrock](https://python.langchain.com/v0.2/docs/integrations/chat/bedrock/)

Confira a documentação mais atualizada para se ter a lista completa: https://python.langchain.com/v0.2/docs/integrations/chat/